In [1]:
import json

with open('bahasa-asr-test.json') as fopen:
    data = json.load(fopen)
    
X, Y = data['X'], data['Y']

In [2]:
import malaya_speech
import speech_recognition as sr

r = sr.Recognizer()

In [3]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [4]:
from tqdm import tqdm

wer, cer = [], []
for i in tqdm(range(len(X))):
    try:
        with sr.AudioFile(X[i]) as source:
            a = r.record(source)

        text = r.recognize_google(a, language = 'ms')
        text = preprocessing_text(text)
        
        wer.append(malaya_speech.metrics.calculate_wer(Y[i], text))
        cer.append(malaya_speech.metrics.calculate_cer(Y[i], text))
    except Exception as e:
        print(e)

100%|██████████| 780/780 [05:33<00:00,  2.34it/s]


In [5]:
import numpy as np

np.mean(wer), np.mean(cer)

(0.16477548774284168, 0.05973209121392649)